# Traffic Sign Classification with Keras

Keras exists to make coding deep neural networks simpler. To demonstrate just how easy it is, you’re going to use Keras to build a convolutional neural network in a few dozen lines of code.

You’ll be connecting the concepts from the previous lessons to the methods that Keras provides.

## Dataset

The network you'll build with Keras is similar to the example that you can find in Keras’s GitHub repository that builds out a [convolutional neural network for MNIST](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py). 

However, instead of using the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset, you're going to use the [German Traffic Sign Recognition Benchmark](http://benchmark.ini.rub.de/?section=gtsrb&subsection=news) dataset that you've used previously.

You can download pickle files with sanitized traffic sign data here.

## Overview

Here are the steps you'll take to build the network:

1. First load the data.
2. Build a feedforward neural network to classify traffic signs.
3. Build a convolutional neural network to classify traffic signs.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

## Load the Data

Start by importing the data from the pickle file.

In [1]:
import math

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
import tensorflow as tf
import pickle 

%matplotlib inline

/home/rbank/.pyenv/versions/3.5.1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
# TODO: Implement load the data here.
training_file = "traffic-signs-data.zip/train.p"
testing_file = "traffic-signs-data.zip/test.p"

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
train.keys()

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train.shape[1:] == (32,32,3)), "The dimensions of the images are not 32 x 32 x 3."

## Normalize the data

Now that you've loaded the training data, normalize the input so that it has a mean of 0 and a range between -0.5 and 0.5.

In [13]:
# TODO: Implement data normalization here.
def normalize(x):
    return x / 255.0 - 0.5


if X_train.max() > 1:
    X_train = normalize(X_train)
    X_test = normalize(X_test)

# STOP: Do not change the tests below. Your implementation should pass these tests. 
assert(round(np.mean(X_train)) == 0), "The mean of the input data is: %f" % np.mean(X_train)
assert(np.min(X_train) == -0.5 and np.max(X_train) == 0.5), "The range of the input data is: %.1f to %.1f" % (np.min(X_train), np.max(X_train))

In [15]:
if False:
    X_train = X_train.reshape((-1, 32*32*3))
    X_test = X_test.reshape((-1, 32*32*3))

## Build a Two-Layer Feedfoward Network

The code you've written so far is for data processing, not specific to Keras. Here you're going to build Keras-specific code.

Build a two-layer feedforward neural network, with 128 neurons in the fully-connected hidden layer. 

To get started, review the Keras documentation about [models](https://keras.io/models/sequential/) and [layers](https://keras.io/layers/core/).

The Keras example of a [Multi-Layer Perceptron](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py) network is similar to what you need to do here. Use that as a guide, but keep in mind that there are a number of differences.

In [16]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

nb_classes = np.unique(y_train).shape[0]
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)


In [7]:
# TODO: Build a two-layer feedforward neural network with Keras here.
# convert class vectors to binary class matrices

model = Sequential()
model.add(Dense(128, input_shape=(32*32*3,), activation='relu', name='hidden1'))
model.add(Dense(nb_classes, activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(model.get_layer(name="hidden1").input_shape == (None, 32*32*3)), "The input shape is: %s" % model.get_layer(name="hidden1").input_shape
assert(model.get_layer(name="output").output_shape == (None, 43)), "The output shape is: %s" % model.get_layer(name="output").output_shape 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
hidden1 (Dense)                  (None, 128)           393344      dense_input_1[0][0]              
____________________________________________________________________________________________________
output (Dense)                   (None, 43)            5547        hidden1[0][0]                    
Total params: 398891
____________________________________________________________________________________________________


## Train the Network
Compile and train the network for 2 epochs. [Use the `adam` optimizer, with `categorical_crossentropy` loss.](https://keras.io/models/sequential/)

Hint 1: In order to use categorical cross entropy, you will need to [one-hot encode the labels](https://github.com/fchollet/keras/blob/master/keras/utils/np_utils.py).

Hint 2: In order to pass the input images to the fully-connected hidden layer, you will need to [reshape the input](https://github.com/fchollet/keras/blob/master/examples/mnist_mlp.py).

Hint 3: Keras's `.fit()` method returns a `History.history` object, which the tests below use. Save that to a variable named `history`.

In [8]:
# TODO: Compile and train the model here.
history = model.fit(X_train, y_train,
                    batch_size=256, nb_epoch=2,
                    verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test,y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['acc'][0] > 0.5), "The training accuracy was: %.3f" % history.history['acc']

Train on 39209 samples, validate on 12630 samples
Epoch 1/2
39209/39209 [==============================] - 3s - loss: 1.8989 - acc: 0.5191 - val_loss: 1.4663 - val_acc: 0.5914
Epoch 2/2
39209/39209 [==============================] - 1s - loss: 0.9129 - acc: 0.7709 - val_loss: 1.1463 - val_acc: 0.6838
Test score: 1.14632386063
Test accuracy: 0.683768804415


## Validate the Network
Split the training data into a training and validation set.

Measure the [validation accuracy](https://keras.io/models/sequential/) of the network after two training epochs.

Hint: [Use the `train_test_split()` method](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) from scikit-learn.

In [17]:
print(X_train.shape, y_train.shape)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)
print(X_train.shape, y_train.shape)

(39209, 32, 32, 3) (39209, 43)
(29406, 32, 32, 3) (29406, 43)


In [11]:
# TODO: Split some of the training data into a validation dataset.
# TODO: Compile and train the model to measure validation accuracy.

history = model.fit(X_train, y_train,
                    batch_size=256, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(round(X_train.shape[0] / float(X_val.shape[0])) == 3), "The training set is %.3f times larger than the validation set." % X_train.shape[0] / float(X_val.shape[0])
assert(history.history['val_acc'][0] > 0.6), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

Train on 29406 samples, validate on 9803 samples
Epoch 1/2
29406/29406 [==============================] - 1s - loss: 0.6579 - acc: 0.8402 - val_loss: 0.5766 - val_acc: 0.8633
Epoch 2/2
29406/29406 [==============================] - 1s - loss: 0.5475 - acc: 0.8650 - val_loss: 0.5466 - val_acc: 0.8451
Test score: 0.939177096617
Test accuracy: 0.730245447357


**Validation Accuracy**: val_acc: 0.7680 Test accuracy: 0.655265241489

## Congratulations
You've built a feedforward neural network in Keras!

Don't stop here! Next, you'll add a convolutional layer to drive.py.

## Convolutions
Build a new network, similar to your existing network. Before the hidden layer, add a 3x3 [convolutional layer](https://keras.io/layers/convolutional/#convolution2d) with 32 filters and valid padding.

Then compile and train the network.

Hint 1: The Keras example of a [convolutional neural network](https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py) for MNIST would be a good example to review.

Hint 2: Now that the first layer of the network is a convolutional layer, you no longer need to reshape the input images before passing them to the network. You might need to reload your training data to recover the original shape.

Hint 3: Add a [`Flatten()` layer](https://keras.io/layers/core/#flatten) between the convolutional layer and the fully-connected hidden layer.

In [24]:
# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.

from keras.layers import Convolution2D, MaxPooling2D

img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)


input_shape = (32, 32, 3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

# model.add(MaxPooling2D(pool_size=pool_size))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128, input_shape=(32*32*3,), activation='relu', name='hidden1'))
model.add(Dense(nb_classes, activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=32, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_7 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_7[0][0]      
____________________________________________________________________________________________________
activation_7 (Activation)        (None, 30, 30, 32)    0           convolution2d_7[0][0]            
____________________________________________________________________________________________________
flatten_7 (Flatten)              (None, 28800)         0           activation_7[0][0]               
____________________________________________________________________________________________________
hidden1 (Dense)                  (None, 128)           3686528     flatten_7[0][0]                  
___________________________________________________________________________________________

**Validation Accuracy**: val_acc: 0.9544 Test accuracy: 0.8813

## Pooling
Re-construct your network and add a 2x2 [pooling layer](https://keras.io/layers/pooling/#maxpooling2d) immediately following your convolutional layer.

Then compile and train the network.

In [25]:
# TODO: Re-construct the network and add a pooling layer after the convolutional layer.
# TODO: Compile and train the model.

# TODO: Re-construct the network and add a convolutional layer before the first fully-connected layer.
# TODO: Compile and train the model.

from keras.layers import Convolution2D, MaxPooling2D

img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)


input_shape = (32, 32, 3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128, input_shape=(32*32*3,), activation='relu', name='hidden1'))
model.add(Dense(nb_classes, activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=32, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_8 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_8[0][0]      
____________________________________________________________________________________________________
maxpooling2d_7 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_8[0][0]            
____________________________________________________________________________________________________
activation_8 (Activation)        (None, 15, 15, 32)    0           maxpooling2d_7[0][0]             
____________________________________________________________________________________________________
flatten_8 (Flatten)              (None, 7200)          0           activation_8[0][0]               
___________________________________________________________________________________________

**Validation Accuracy**:  val_acc: 0.9523 Test accuracy: 0.8762

## Dropout
Re-construct your network and add [dropout](https://keras.io/layers/core/#dropout) after the pooling layer. Set the dropout rate to 50%.

In [26]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(p=0.5))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dense(128, input_shape=(32*32*3,), activation='relu', name='hidden1'))
model.add(Dense(nb_classes, activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=32, nb_epoch=2,
                    verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])


# STOP: Do not change the tests below. Your implementation should pass these tests.
assert(history.history['val_acc'][0] > 0.9), "The validation accuracy is: %.3f" % history.history['val_acc'][0]

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_9 (Convolution2D)  (None, 30, 30, 32)    896         convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
maxpooling2d_8 (MaxPooling2D)    (None, 15, 15, 32)    0           convolution2d_9[0][0]            
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 15, 15, 32)    0           maxpooling2d_8[0][0]             
____________________________________________________________________________________________________
activation_9 (Activation)        (None, 15, 15, 32)    0           dropout_1[0][0]                  
___________________________________________________________________________________________

**Validation Accuracy**: val_acc: 0.9359 Test accuracy: 0.8669

## Optimization
Congratulations! You've built a neural network with convolutions, pooling, dropout, and fully-connected layers, all in just a few lines of code.

Have fun with the model and see how well you can do! Add more layers, or regularization, or different padding, or batches, or more training epochs.

What is the best validation accuracy you can achieve?

In [48]:
# TODO: Re-construct the network and add dropout after the pooling layer.
# TODO: Compile and train the model.

model = Sequential()
# model.add(Dropout(p=0.8, input_shape=input_shape))

model.add(Convolution2D(32, 3, 3, input_shape=input_shape,
                        border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(p=0.8))

# model.add(Convolution2D(64, 3, 3,
#                         border_mode='valid', activation='relu'))
# model.add(MaxPooling2D(pool_size=pool_size))
# model.add(Dropout(p=0.8))


model.add(Flatten())
model.add(Dense(256, activation='relu', name='hidden1'))
# model.add(Dense(128, activation='relu', name='hidden2'))

model.add(Dense(nb_classes, activation='softmax', name='output'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    batch_size=128, nb_epoch=25,
                    verbose=1, validation_data=(X_val, y_val))

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_42 (Convolution2D) (None, 30, 30, 32)    896         convolution2d_input_16[0][0]     
____________________________________________________________________________________________________
maxpooling2d_38 (MaxPooling2D)   (None, 15, 15, 32)    0           convolution2d_42[0][0]           
____________________________________________________________________________________________________
dropout_41 (Dropout)             (None, 15, 15, 32)    0           maxpooling2d_38[0][0]            
____________________________________________________________________________________________________
flatten_25 (Flatten)             (None, 7200)          0           dropout_41[0][0]                 
___________________________________________________________________________________________

**Best Validation Accuracy:** 98.8%

## Testing
Once you've picked out your best model, it's time to test it.

Load up the test data and use the [`evaluate()` method](https://keras.io/models/model/#evaluate) to see how well it does.

Hint 1: After you load your test data, don't forget to normalize the input and one-hot encode the output, so it matches the training data.

Hint 2: The `evaluate()` method should return an array of numbers. Use the `metrics_names()` method to get the labels.

In [ ]:
# with open('./test.p', mode='rb') as f:
#     test = pickle.load(f)
    
# X_test = test['features']
# y_test = test['labels']
# X_test = X_test.astype('float32')
# X_test /= 255
# X_test -= 0.5
# Y_test = np_utils.to_categorical(y_test, 43)

# model.evaluate(X_test, Y_test)

In [49]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
print(model.metrics_names)

Test score: 0.429562601623
Test accuracy: 0.902533650106
['loss', 'acc']


**Test Accuracy:** 0.9087

## Summary
Keras is a great tool to use if you want to quickly build a neural network and evaluate performance.